In [1]:
%cd ../
%matplotlib inline

/home/zulqarnain/develop/htfatorch


In [2]:
import collections
import logging
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils
import glob
import os

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
haxby_dir = '/home/zulqarnain/fmri_data/Haxby_Final/'

task_log_csvs = 'events_files'

HAXBY_FILENAME_TEMPLATE = 'sub%d_run%02d_mc_MNI_masked.nii.gz'
TASK_FILE_TEMPLATE = 'sub-%d_task-objectviewing_run-%02d_events.tsv'

HAXBY_FILE = 'data/haxby_mc_zrest.db'

TASK_ONSET_DELAY = 3
def haxby_filename(subject, run):
    return haxby_dir + (HAXBY_FILENAME_TEMPLATE % (subject, run))


class TaskElement:
    def __init__(self, task, start, end,run,t_seconds=True):
        def round_off_time(t):
            if t is not None:
                if task != 'rest':
                    return round((t + TASK_ONSET_DELAY) / 2.5)
                else:
                    return round(t)
            else:
                return None

        self.task = task
        self.run = run
        if t_seconds:
            self.start_time = round_off_time(start)
            self.end_time = round_off_time(end)
        else:
            self.start_time = start
            self.end_time = end

def parse_task_lines(lines,run):
    for (i, line) in enumerate(lines):
        cols = line.strip('\n').split('\t')
        task = cols[2]
        start_time = float(cols[0])
        end_time = start_time + float(cols[1])
        yield TaskElement(task, start_time, end_time,run)

def make_task_blocks(tasks):
    current_task = tasks[0].task
    current_start = tasks[0].start_time
    run = tasks[0].run
    for i in range(1,len(tasks)):
        task_name = tasks[i].task
        if task_name == current_task:
            end_time = tasks[i].end_time
        else:
            yield TaskElement(current_task,current_start,end_time, run, t_seconds=False)
            current_task = task_name
            current_start = tasks[i].start_time
    yield TaskElement(current_task,current_start,end_time, run, t_seconds=False)

def rest_tasks(tasks):
    run = tasks[0].run
    yield TaskElement('rest', 0, tasks[0].start_time - 1, run, t_seconds=False)
    for i in range(1, len(tasks)):
        rest_start = tasks[i - 1].end_time + 1
        rest_end = tasks[i].start_time - 1
        yield TaskElement('rest', rest_start, rest_end, run, t_seconds=False)
    yield TaskElement('rest', tasks[-1].end_time + 1, None, run, t_seconds=False)


bad_runs = collections.defaultdict(lambda: [])
bad_runs[5] = [9]
bad_runs[1] = [11]
def read_tasks(task_csv):
    def sentinel(f):
        return f if f is not None else 0.0

    with open(task_csv, 'r') as task_csv_file:
        header = task_csv_file.readline().strip('\n').split('\t')
        filename = task_csv.split('/')[-1]
        subject = int(filename.split('-')[1][0])
        run = int(filename.split('-')[-1][:2])
        logging.info('Subject %d', subject)
        task_lines = list(parse_task_lines(task_csv_file.readlines(),run=run))
        task_lines = list(make_task_blocks(task_lines))
        task_lines += list(rest_tasks(task_lines))
        rest_lines = [r for r in task_lines if r.task == 'rest']
        rest_lines = sorted(rest_lines, key=lambda t: sentinel(t.run))
        rest_starts_dict = {key: [] for key in range(1, 13)}
        rest_ends_dict = {key: [] for key in range(1, 13)}
        for (i, rest) in enumerate(rest_lines):
            if rest.run in bad_runs[subject]:
                continue
            if rest.end_time is not None and rest.start_time is not None:
                rest_ends_dict[rest.run].append(rest.end_time)
                rest_starts_dict[rest.run].append(rest.start_time)
        task_lines = sorted(task_lines, key=lambda t: sentinel(t.start_time))
        for (i, task) in enumerate(task_lines):
            if task.run in bad_runs[subject]:
                logging.info('Excluding block %d %s of run %d for bad data', i,
                             task.task, task.run)
                continue
            logging.info('Block %d %s of run %d started at %f, ended at %f', i,
                         task.task, task.run, sentinel(task.start_time), sentinel(task.end_time))
            result = niidb.FMriActivationBlock(zscore=True, zscore_by_rest=True)
            result.subject = subject
            result.task = task.task
            result.run = task.run
            result.block = i
            result.start_time = task.start_time
            result.end_time = task.end_time
            result.rest_start_times = rest_starts_dict[result.run]
            result.rest_end_times = rest_ends_dict[result.run]
            yield result

In [5]:
OVERRIDE = True

if not os.path.exists(HAXBY_FILE) or OVERRIDE:
    haxby_db = niidb.FMriActivationsDb(HAXBY_FILE, mask='/home/zulqarnain/fmri_data/Haxby_Final/mask/haxby_mask.nii.gz',
                                         smooth=6)

    for task_csv in utils.sorted_glob(haxby_dir + task_log_csvs + '/*.tsv'):
        for block in read_tasks(task_csv):
            block.filename = haxby_filename(block.subject, block.run)
#             block.mask = mask_filename(block.subject, block.run)
            haxby_db.upsert(block)

01/24/2020 18:40:23 Subject 1
01/24/2020 18:40:23 Block 0 rest of run 1 started at 0.000000, ended at 5.000000
01/24/2020 18:40:23 Block 1 scissors of run 1 started at 6.000000, ended at 15.000000
01/24/2020 18:40:23 Block 2 rest of run 1 started at 16.000000, ended at 19.000000
01/24/2020 18:40:23 Block 3 face of run 1 started at 20.000000, ended at 29.000000
01/24/2020 18:40:23 Block 4 rest of run 1 started at 30.000000, ended at 34.000000
01/24/2020 18:40:23 Block 5 cat of run 1 started at 35.000000, ended at 44.000000
01/24/2020 18:40:23 Block 6 rest of run 1 started at 45.000000, ended at 48.000000
01/24/2020 18:40:23 Block 7 shoe of run 1 started at 49.000000, ended at 58.000000
01/24/2020 18:40:23 Block 8 rest of run 1 started at 59.000000, ended at 63.000000
01/24/2020 18:40:23 Block 9 house of run 1 started at 64.000000, ended at 73.000000
01/24/2020 18:40:23 Block 10 rest of run 1 started at 74.000000, ended at 77.000000
01/24/2020 18:40:23 Block 11 scrambledpix of run 1 star

01/24/2020 18:40:24 Block 11 cat of run 6 started at 78.000000, ended at 87.000000
01/24/2020 18:40:24 Block 12 rest of run 6 started at 88.000000, ended at 91.000000
01/24/2020 18:40:24 Block 13 bottle of run 6 started at 92.000000, ended at 101.000000
01/24/2020 18:40:24 Block 14 rest of run 6 started at 102.000000, ended at 106.000000
01/24/2020 18:40:24 Block 15 scissors of run 6 started at 107.000000, ended at 116.000000
01/24/2020 18:40:24 Block 16 rest of run 6 started at 117.000000, ended at 0.000000
01/24/2020 18:40:24 Subject 1
01/24/2020 18:40:24 Block 0 rest of run 7 started at 0.000000, ended at 5.000000
01/24/2020 18:40:24 Block 1 face of run 7 started at 6.000000, ended at 15.000000
01/24/2020 18:40:24 Block 2 rest of run 7 started at 16.000000, ended at 19.000000
01/24/2020 18:40:24 Block 3 chair of run 7 started at 20.000000, ended at 29.000000
01/24/2020 18:40:24 Block 4 rest of run 7 started at 30.000000, ended at 34.000000
01/24/2020 18:40:24 Block 5 scissors of run

01/24/2020 18:40:24 Block 8 rest of run 12 started at 59.000000, ended at 63.000000
01/24/2020 18:40:24 Block 9 face of run 12 started at 64.000000, ended at 73.000000
01/24/2020 18:40:24 Block 10 rest of run 12 started at 74.000000, ended at 77.000000
01/24/2020 18:40:24 Block 11 shoe of run 12 started at 78.000000, ended at 87.000000
01/24/2020 18:40:24 Block 12 rest of run 12 started at 88.000000, ended at 91.000000
01/24/2020 18:40:24 Block 13 cat of run 12 started at 92.000000, ended at 101.000000
01/24/2020 18:40:24 Block 14 rest of run 12 started at 102.000000, ended at 106.000000
01/24/2020 18:40:24 Block 15 scissors of run 12 started at 107.000000, ended at 116.000000
01/24/2020 18:40:24 Block 16 rest of run 12 started at 117.000000, ended at 0.000000
01/24/2020 18:40:24 Subject 2
01/24/2020 18:40:24 Block 0 rest of run 1 started at 0.000000, ended at 5.000000
01/24/2020 18:40:24 Block 1 scissors of run 1 started at 6.000000, ended at 15.000000
01/24/2020 18:40:24 Block 2 rest

01/24/2020 18:40:25 Block 1 cat of run 6 started at 6.000000, ended at 15.000000
01/24/2020 18:40:25 Block 2 rest of run 6 started at 16.000000, ended at 19.000000
01/24/2020 18:40:25 Block 3 scrambledpix of run 6 started at 20.000000, ended at 29.000000
01/24/2020 18:40:25 Block 4 rest of run 6 started at 30.000000, ended at 34.000000
01/24/2020 18:40:25 Block 5 scissors of run 6 started at 35.000000, ended at 44.000000
01/24/2020 18:40:25 Block 6 rest of run 6 started at 45.000000, ended at 48.000000
01/24/2020 18:40:25 Block 7 chair of run 6 started at 49.000000, ended at 58.000000
01/24/2020 18:40:25 Block 8 rest of run 6 started at 59.000000, ended at 63.000000
01/24/2020 18:40:25 Block 9 bottle of run 6 started at 64.000000, ended at 73.000000
01/24/2020 18:40:25 Block 10 rest of run 6 started at 74.000000, ended at 77.000000
01/24/2020 18:40:25 Block 11 shoe of run 6 started at 78.000000, ended at 87.000000
01/24/2020 18:40:25 Block 12 rest of run 6 started at 88.000000, ended a

01/24/2020 18:40:26 Block 11 house of run 11 started at 78.000000, ended at 87.000000
01/24/2020 18:40:26 Block 12 rest of run 11 started at 88.000000, ended at 91.000000
01/24/2020 18:40:26 Block 13 face of run 11 started at 92.000000, ended at 101.000000
01/24/2020 18:40:26 Block 14 rest of run 11 started at 102.000000, ended at 106.000000
01/24/2020 18:40:26 Block 15 scissors of run 11 started at 107.000000, ended at 116.000000
01/24/2020 18:40:26 Block 16 rest of run 11 started at 117.000000, ended at 0.000000
01/24/2020 18:40:26 Subject 2
01/24/2020 18:40:26 Block 0 rest of run 12 started at 0.000000, ended at 5.000000
01/24/2020 18:40:26 Block 1 bottle of run 12 started at 6.000000, ended at 15.000000
01/24/2020 18:40:26 Block 2 rest of run 12 started at 16.000000, ended at 19.000000
01/24/2020 18:40:26 Block 3 house of run 12 started at 20.000000, ended at 29.000000
01/24/2020 18:40:26 Block 4 rest of run 12 started at 30.000000, ended at 34.000000
01/24/2020 18:40:26 Block 5 ch

...

01/24/2020 18:40:28 Block 1 house of run 10 started at 6.000000, ended at 15.000000
01/24/2020 18:40:28 Block 2 rest of run 10 started at 16.000000, ended at 19.000000
01/24/2020 18:40:28 Block 3 scissors of run 10 started at 20.000000, ended at 29.000000
01/24/2020 18:40:28 Block 4 rest of run 10 started at 30.000000, ended at 34.000000
01/24/2020 18:40:28 Block 5 bottle of run 10 started at 35.000000, ended at 44.000000
01/24/2020 18:40:28 Block 6 rest of run 10 started at 45.000000, ended at 48.000000
01/24/2020 18:40:28 Block 7 face of run 10 started at 49.000000, ended at 58.000000
01/24/2020 18:40:28 Block 8 rest of run 10 started at 59.000000, ended at 63.000000
01/24/2020 18:40:28 Block 9 chair of run 10 started at 64.000000, ended at 73.000000
01/24/2020 18:40:28 Block 10 rest of run 10 started at 74.000000, ended at 77.000000
01/24/2020 18:40:28 Block 11 shoe of run 10 started at 78.000000, ended at 87.000000
01/24/2020 18:40:28 Block 12 rest of run 10 started at 88.000000, e

01/24/2020 18:40:29 Block 11 house of run 3 started at 78.000000, ended at 87.000000
01/24/2020 18:40:29 Block 12 rest of run 3 started at 88.000000, ended at 91.000000
01/24/2020 18:40:29 Block 13 face of run 3 started at 92.000000, ended at 101.000000
01/24/2020 18:40:29 Block 14 rest of run 3 started at 102.000000, ended at 106.000000
01/24/2020 18:40:29 Block 15 scissors of run 3 started at 107.000000, ended at 116.000000
01/24/2020 18:40:29 Block 16 rest of run 3 started at 117.000000, ended at 0.000000
01/24/2020 18:40:29 Subject 5
01/24/2020 18:40:29 Block 0 rest of run 4 started at 0.000000, ended at 5.000000
01/24/2020 18:40:29 Block 1 face of run 4 started at 6.000000, ended at 15.000000
01/24/2020 18:40:29 Block 2 rest of run 4 started at 16.000000, ended at 19.000000
01/24/2020 18:40:29 Block 3 scrambledpix of run 4 started at 20.000000, ended at 29.000000
01/24/2020 18:40:29 Block 4 rest of run 4 started at 30.000000, ended at 34.000000
01/24/2020 18:40:29 Block 5 scissors

01/24/2020 18:40:30 Excluding block 5 chair of run 9 for bad data
01/24/2020 18:40:30 Excluding block 6 rest of run 9 for bad data
01/24/2020 18:40:30 Excluding block 7 scrambledpix of run 9 for bad data
01/24/2020 18:40:30 Excluding block 8 rest of run 9 for bad data
01/24/2020 18:40:30 Excluding block 9 face of run 9 for bad data
01/24/2020 18:40:30 Excluding block 10 rest of run 9 for bad data
01/24/2020 18:40:30 Excluding block 11 shoe of run 9 for bad data
01/24/2020 18:40:30 Excluding block 12 rest of run 9 for bad data
01/24/2020 18:40:30 Excluding block 13 cat of run 9 for bad data
01/24/2020 18:40:30 Excluding block 14 rest of run 9 for bad data
01/24/2020 18:40:30 Excluding block 15 scissors of run 9 for bad data
01/24/2020 18:40:30 Excluding block 16 rest of run 9 for bad data
01/24/2020 18:40:30 Subject 5
01/24/2020 18:40:30 Block 0 rest of run 10 started at 0.000000, ended at 5.000000
01/24/2020 18:40:30 Block 1 shoe of run 10 started at 6.000000, ended at 15.000000
01/24/

01/24/2020 18:40:30 Block 1 house of run 4 started at 6.000000, ended at 15.000000
01/24/2020 18:40:30 Block 2 rest of run 4 started at 16.000000, ended at 19.000000
01/24/2020 18:40:30 Block 3 scissors of run 4 started at 20.000000, ended at 29.000000
01/24/2020 18:40:30 Block 4 rest of run 4 started at 30.000000, ended at 34.000000
01/24/2020 18:40:30 Block 5 bottle of run 4 started at 35.000000, ended at 44.000000
01/24/2020 18:40:30 Block 6 rest of run 4 started at 45.000000, ended at 48.000000
01/24/2020 18:40:30 Block 7 face of run 4 started at 49.000000, ended at 58.000000
01/24/2020 18:40:30 Block 8 rest of run 4 started at 59.000000, ended at 63.000000
01/24/2020 18:40:30 Block 9 chair of run 4 started at 64.000000, ended at 73.000000
01/24/2020 18:40:30 Block 10 rest of run 4 started at 74.000000, ended at 77.000000
01/24/2020 18:40:30 Block 11 shoe of run 4 started at 78.000000, ended at 87.000000
01/24/2020 18:40:30 Block 12 rest of run 4 started at 88.000000, ended at 91.0

01/24/2020 18:40:31 Block 12 rest of run 9 started at 88.000000, ended at 91.000000
01/24/2020 18:40:31 Block 13 face of run 9 started at 92.000000, ended at 101.000000
01/24/2020 18:40:31 Block 14 rest of run 9 started at 102.000000, ended at 106.000000
01/24/2020 18:40:31 Block 15 scissors of run 9 started at 107.000000, ended at 116.000000
01/24/2020 18:40:31 Block 16 rest of run 9 started at 117.000000, ended at 0.000000
01/24/2020 18:40:31 Subject 6
01/24/2020 18:40:31 Block 0 rest of run 10 started at 0.000000, ended at 5.000000
01/24/2020 18:40:31 Block 1 house of run 10 started at 6.000000, ended at 15.000000
01/24/2020 18:40:31 Block 2 rest of run 10 started at 16.000000, ended at 19.000000
01/24/2020 18:40:31 Block 3 scrambledpix of run 10 started at 20.000000, ended at 29.000000
01/24/2020 18:40:31 Block 4 rest of run 10 started at 30.000000, ended at 34.000000
01/24/2020 18:40:31 Block 5 face of run 10 started at 35.000000, ended at 44.000000
01/24/2020 18:40:31 Block 6 res

In [6]:
logging.info('Finished building NiiDb out of AffVids dataset in %s', haxby_dir)

01/24/2020 18:40:31 Finished building NiiDb out of AffVids dataset in /home/zulqarnain/fmri_data/Haxby_Final/
